In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
import xml.etree.ElementTree as ET

In [6]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename

In [7]:
import re

In [8]:
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
file = askopenfilename() # show an "Open" dialog box and return the path to the selected file

In [9]:
parseComments = ET.XMLParser(target=ET.TreeBuilder(insert_comments=True))

In [10]:
tree = ET.parse(file, parseComments)

In [11]:
root = tree.getroot()

In [12]:
xmlMap = {c: p for p in tree.iter() for c in p }

In [13]:
nextHasComment = False

In [15]:
class Depth:                     # The target object of the parser
    maxDepth = 0
    depth = 0
    def start(self, tag, attrib):   # Called for each opening tag.
        self.depth += 1
        if self.depth > self.maxDepth:
            self.maxDepth = self.depth
    def end(self, tag):             # Called for each closing tag.
        self.depth -= 1
    def data(self, data):
        pass            # We do not need to do anything with data.
    def close(self):    # Called when all data has been parsed.
        return self.maxDepth

In [16]:
parseDepth = ET.XMLParser(target=Depth())
parseDepth.feed(ET.tostring(root))
maxDepth = parseDepth.close()

In [17]:
def getText(node):
    try:
        match = re.search('(?:{.*?})(.*)', node)
        text = match.group(1)
    except:
        text = node
    return text

In [18]:
def getColumn(node):
    counter = 0
    newValue = node
    while newValue != None:
        try:
            newValue = xmlMap.get(newValue)
            counter += 1
        except:
            break
    return counter

In [19]:
for index, node in enumerate(root.iter()):
    nodeText = getText(node.tag)
    column = getColumn(node)
    neededColumns = maxDepth + 1
    
    if "function Comment" in str(node):
        nodeComment = node.text[:-1]
        nextHasComment = True
    elif index == 0:
        title = nodeText
        df = pd.DataFrame(columns=['Column {}'.format(x) for x in list(range(1,neededColumns))])
        defaultColumns = ['Annotation', 'Description']
        df[defaultColumns] = None
        df.style.set_caption(title)
        df.loc[len(df)] = {'Column {}'.format(column): nodeText, 'Annotation' : "", 'Description' : ""}
    else:
        df.loc[len(df)] = {'Column {}'.format(column): nodeText, 'Annotation' : "("+nodeComment+")", 'Description' : ""} \
        if (nextHasComment and index != 0) else {'Column {}'.format(column): nodeText, 'Annotation' : "", 'Description' : ""}
        nextHasComment = False

In [ ]:
df.to_excel

In [20]:
from tkinter.filedialog import asksaveasfile

In [22]:
exportName = 'test'

In [27]:
def file_save(exportName):
    #Tk().withdraw()
    f =  asksaveasfile(title=exportName, defaultextension=".xml")
    if f is None: # asksaveasfile return `None` if dialog closed with "cancel".
        return
    #text2save = str(text.get(1.0, END)) # starts from `1.0`, not `0.0`
    f.write(filedialog)
    f.close()

In [28]:
file_save(exportName)